In [49]:
import pandas as pd
import re
import numpy as np

In [10]:
pd.set_option('display.max_columns', None)

In [5]:
tracking_df = pd.read_csv('../data/agg_data/tracking_df.csv', index_col=0, header=0)

In [6]:
tracking_df.head()

,gameId,playId,nflId,displayName,frameId,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event
0,2022100600,90,33084.0,Matt Ryan,1,2022-10-06 20:17:04.799999,2.0,IND,left,90.42,23.74,0.11,0.04,0.03,271.98,257.76,NaN
1,2022100600,90,33084.0,Matt Ryan,2,2022-10-06 20:17:04.900000,2.0,IND,left,90.39,23.74,0.14,0.06,0.03,272.84,256.68,NaN
2,2022100600,90,33084.0,Matt Ryan,3,2022-10-06 20:17:05.000000,2.0,IND,left,90.36,23.73,0.17,0.09,0.03,272.84,254.91,NaN
3,2022100600,90,33084.0,Matt Ryan,4,2022-10-06 20:17:05.099999,2.0,IND,left,90.32,23.73,0.19,0.11,0.04,275.80,260.06,NaN
4,2022100600,90,33084.0,Matt Ryan,5,2022-10-06 20:17:05.200000,2.0,IND,left,90.28,23.72,0.20,0.13,0.04,275.80,257.79,NaN


In [7]:
plays = pd.read_csv('../data/provided_data/plays.csv')

In [11]:
plays.head()

,gameId,playId,ballCarrierId,ballCarrierDisplayName,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,passResult,passLength,penaltyYards,prePenaltyPlayResult,playResult,playNullifiedByPenalty,absoluteYardlineNumber,offenseFormation,defendersInTheBox,passProbability,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPoints,expectedPointsAdded,foulName1,foulName2,foulNFLId1,foulNFLId2
0,2022100908,3537,48723,Parker Hesse,(7:52) (Shotgun) M.Mariota pass short middle t...,4,1,10,ATL,TB,ATL,41,7:52,21,7,C,6.0,NaN,9,9,N,69,SHOTGUN,7.0,0.747284,0.976785,0.023215,-0.006110,0.006110,2.360609,0.981955,NaN,NaN,NaN,NaN
1,2022091103,3126,52457,Chase Claypool,(7:38) (Shotgun) C.Claypool right end to PIT 3...,4,1,10,PIT,CIN,PIT,34,7:38,14,20,NaN,NaN,NaN,3,3,N,76,SHOTGUN,7.0,0.416454,0.160485,0.839515,-0.010865,0.010865,1.733344,-0.263424,NaN,NaN,NaN,NaN
2,2022091111,1148,42547,Darren Waller,(8:57) D.Carr pass short middle to D.Waller to...,2,2,5,LV,LAC,LV,30,8:57,10,3,C,11.0,NaN,15,15,N,40,I_FORM,6.0,0.267933,0.756661,0.243339,-0.037409,0.037409,1.312855,1.133666,NaN,NaN,NaN,NaN
3,2022100212,2007,46461,Mike Boone,(13:12) M.Boone left tackle to DEN 44 for 7 ya...,3,2,10,DEN,LV,DEN,37,13:12,19,16,NaN,NaN,NaN,7,7,N,47,SINGLEBACK,6.0,0.592704,0.620552,0.379448,-0.002451,0.002451,1.641006,-0.043580,NaN,NaN,NaN,NaN
4,2022091900,1372,47857,Devin Singletary,(8:33) D.Singletary right guard to TEN 32 for ...,2,1,10,BUF,TEN,TEN,35,8:33,7,7,NaN,NaN,NaN,3,3,N,75,I_FORM,7.0,0.470508,0.836290,0.163710,0.001053,-0.001053,3.686428,-0.167903,NaN,NaN,NaN,NaN


In [31]:
tackles = pd.read_csv('../data/provided_data/tackles.csv')

In [32]:
tackles.head()

,gameId,playId,nflId,tackle,assist,forcedFumble,pff_missedTackle
0,2022090800,101,42816,1,0,0,0
1,2022090800,393,46232,1,0,0,0
2,2022090800,486,40166,1,0,0,0
3,2022090800,646,47939,1,0,0,0
4,2022090800,818,40107,1,0,0,0


In [35]:
def make_gameplay(gameid, playid):
    return '_'.join([str(gameid), str(playid)])

In [36]:


game_play_tackles = tackles.apply(lambda x: make_gameplay(x['gameId'], x['playId']), axis = 1)


print(len(set(game_play_tackles)))

tackles.insert(loc = 0, value = game_play_tackles, column = 'gameplayId')

12025


In [37]:
tackles.head()

,gameplayId,gameId,playId,nflId,tackle,assist,forcedFumble,pff_missedTackle
0,2022090800_101,2022090800,101,42816,1,0,0,0
1,2022090800_393,2022090800,393,46232,1,0,0,0
2,2022090800_486,2022090800,486,40166,1,0,0,0
3,2022090800_646,2022090800,646,47939,1,0,0,0
4,2022090800_818,2022090800,818,40107,1,0,0,0


In [18]:
tackles.shape

(17426, 7)

In [40]:
game_play_plays = plays.apply(lambda x: make_gameplay(x['gameId'], x['playId']), axis = 1)

print(len(set(game_play_plays)))

plays.insert(loc = 0, value = game_play_plays, column = 'gameplayId')

12486


In [41]:
plays.head()

,gameplayId,gameId,playId,ballCarrierId,ballCarrierDisplayName,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,passResult,passLength,penaltyYards,prePenaltyPlayResult,playResult,playNullifiedByPenalty,absoluteYardlineNumber,offenseFormation,defendersInTheBox,passProbability,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPoints,expectedPointsAdded,foulName1,foulName2,foulNFLId1,foulNFLId2
0,2022100908_3537,2022100908,3537,48723,Parker Hesse,(7:52) (Shotgun) M.Mariota pass short middle t...,4,1,10,ATL,TB,ATL,41,7:52,21,7,C,6.0,NaN,9,9,N,69,SHOTGUN,7.0,0.747284,0.976785,0.023215,-0.006110,0.006110,2.360609,0.981955,NaN,NaN,NaN,NaN
1,2022091103_3126,2022091103,3126,52457,Chase Claypool,(7:38) (Shotgun) C.Claypool right end to PIT 3...,4,1,10,PIT,CIN,PIT,34,7:38,14,20,NaN,NaN,NaN,3,3,N,76,SHOTGUN,7.0,0.416454,0.160485,0.839515,-0.010865,0.010865,1.733344,-0.263424,NaN,NaN,NaN,NaN
2,2022091111_1148,2022091111,1148,42547,Darren Waller,(8:57) D.Carr pass short middle to D.Waller to...,2,2,5,LV,LAC,LV,30,8:57,10,3,C,11.0,NaN,15,15,N,40,I_FORM,6.0,0.267933,0.756661,0.243339,-0.037409,0.037409,1.312855,1.133666,NaN,NaN,NaN,NaN
3,2022100212_2007,2022100212,2007,46461,Mike Boone,(13:12) M.Boone left tackle to DEN 44 for 7 ya...,3,2,10,DEN,LV,DEN,37,13:12,19,16,NaN,NaN,NaN,7,7,N,47,SINGLEBACK,6.0,0.592704,0.620552,0.379448,-0.002451,0.002451,1.641006,-0.043580,NaN,NaN,NaN,NaN
4,2022091900_1372,2022091900,1372,47857,Devin Singletary,(8:33) D.Singletary right guard to TEN 32 for ...,2,1,10,BUF,TEN,TEN,35,8:33,7,7,NaN,NaN,NaN,3,3,N,75,I_FORM,7.0,0.470508,0.836290,0.163710,0.001053,-0.001053,3.686428,-0.167903,NaN,NaN,NaN,NaN


In [47]:
bool(re.search(string = 'no', pattern = 'injur'))

False

In [54]:
list(np.where([True, False, True])[0])

[0, 2]

In [55]:
# is injury data included in the playDescriptions?

# get indices of all plays that have injury in playDescription
injury_plays = list(np.where(plays['playDescription'].apply(lambda x: bool(re.search(pattern = 'injur', string = x))))[0])

In [57]:
print(len(injury_plays))

303


In [61]:
injury_play_descriptions = plays['playDescription'][injury_plays].reset_index(drop = True)

In [102]:
inj_play_dict = dict(zip(plays['gameplayId'][injury_plays], plays['playDescription'][injury_plays]))

In [103]:
inj_play_dict

{'2022091900_1372': '(8:33) D.Singletary right guard to TEN 32 for 3 yards (N.Jones; O.Adeniyi). TEN-O.Adeniyi was injured during the play. His return is Questionable.  O.Adeniyi walks off.',
 '2022100207_2933': '(13:31) (Shotgun) R.Tannehill scrambles up the middle to TEN 28 for 3 yards (Z.Franklin). IND-T.Lewis was injured during the play. TEN-T.Burks was injured during the play. He is Out.',
 '2022102308_588': '(3:20) (Shotgun) D.Carr pass short left to A.Abdullah to LV 23 for 5 yards (C.Kirksey). HST-M.Collins was injured during the play. His return is Questionable.',
 '2022092502_2514': '(7:26) J.Taylor up the middle to IND 37 for 5 yards (N.Bolton, J.Reid). KC-C.Jones was injured during the play.',
 '2022103000_1419': '(8:09) (Shotgun) R.Wilson pass short middle to J.Jeudy to DEN 46 for 8 yards (R.Jenkins). JAX-R.Jenkins was injured during the play. His return is Probable.',
 '2022100600_111': '(14:13) (Shotgun) M.Ryan pass short left to N.Hines to IND 33 for 5 yards (Dj.Jones; N

In [104]:
# create a dictionary of all nflIds involved in the tackling (tackle itself or missed tackle) for each gameplayId
# we'll use these nflIds to link players' names in injury dict to tackling df

# we could eventually expand this dict out to a dataframe where each row is a gameplayId and there are enough columns to accommodate the most tacklers on any given play
tackles_involved_dict = {}

for rownum, row in tackles.iterrows():
    # print(rownum)
    # print(row['gameplayId'])
    tackles_involved_dict.setdefault(row['gameplayId'], []).append(row['nflId'])

In [105]:
tackles_involved_dict

{'2022090800_101': [42816],
 '2022090800_393': [46232],
 '2022090800_486': [40166],
 '2022090800_646': [47939],
 '2022090800_818': [40107, 54650],
 '2022090800_1286': [44976],
 '2022090800_1736': [44976],
 '2022090800_2860': [46085],
 '2022090800_3092': [46190],
 '2022090800_3145': [52647],
 '2022090800_3362': [46190],
 '2022090800_3383': [54488],
 '2022091100_1260': [44851, 44862],
 '2022091100_1672': [40017],
 '2022091100_2114': [53469],
 '2022091100_2353': [38607],
 '2022091100_2512': [53611],
 '2022091100_2640': [44862],
 '2022091100_2878': [53489],
 '2022091100_2899': [37097, 43409],
 '2022091100_3578': [52542],
 '2022091100_3647': [53469],
 '2022091100_3778': [53489],
 '2022091101_1691': [53481, 46073],
 '2022091101_1744': [44903],
 '2022091101_1785': [52452],
 '2022091101_2051': [46527],
 '2022091101_3221': [47799],
 '2022091102_86': [44938],
 '2022091102_531': [52571, 44925],
 '2022091102_756': [53609],
 '2022091102_1493': [54504],
 '2022091102_2065': [54504, 54513],
 '20220911

In [87]:
def find_injured_player_name(inj_str):
    return(re.findall(string = inj_str, pattern = '.* (.*) was injured.*')[0])

In [106]:
injured_players = []
for desc in inj_play_dict.values():
    try:
        injured_players.append(find_injured_player_name(desc))
    except IndexError as e:
        injured_players.append(np.nan)

In [107]:
injured_players

['TEN-O.Adeniyi',
 'TEN-T.Burks',
 'HST-M.Collins',
 'KC-C.Jones',
 'JAX-R.Jenkins',
 'IND-N.Hines',
 'TB-M.Edwards',
 'DAL-C.McGovern',
 'BLT-M.Moses',
 'DAL-Z.Martin',
 'TB-C.Brate',
 'ATL-A.Ogundeji',
 'NYG-L.Williams',
 'ATL-A.Terrell',
 'BUF-J.Kumerow',
 'PIT-J.Pierre',
 'WAS-J.Allen',
 'CAR-D.Jackson',
 'WAS-D.Forrest',
 'BUF-S.Diggs',
 'BLT-S.Means',
 'TEN-A.Hooker',
 'WAS-D.Payne',
 'BUF-C.Benford',
 'ARZ-M.Wilson',
 'CIN-J.Williams',
 'TEN-K.Fulton',
 'TB-C.Davis',
 'SF-B.Aiyuk',
 'NYG-N.Williams',
 'NE-D.Harris',
 'MIN-Z.Smith',
 'BLT-J.Bynes',
 'IND-J.Blackmon',
 'BUF-D.Jackson',
 'NYJ-K.Alexander',
 'PIT-C.Heyward',
 'NYG-A.Jackson',
 'NYJ-J.Johnson',
 'ATL-M.Walker',
 'DET-D.Elliott',
 'DAL-E.Elliott',
 'PIT-M.Cole',
 'LA-J.Noteboom',
 'GB-E.Stokes',
 'PHI-F.Cox',
 'CIN-L.Wilson',
 'TEN-C.Jackson',
 'CIN-T.Higgins',
 'IND-B.Cowart',
 'BLT-M.Moses',
 'GB-J.Runyan',
 'BUF-K.Elam',
 'PHI-C.Gardner-Johnson',
 'LV-J.Eluemunor',
 'CAR-D.Brown',
 'NYJ-B.Berrios',
 'MIA-R.Hunt',
 

In [108]:
bad_gameplay_id_inds = list(np.where([pd.isnull(val) for val in injured_players])[0])
print([list(inj_play_dict.values())[id] for id in bad_gameplay_id_inds], end = '\n')

# only the first of the four here is an actual injury, the rest are updates
# manual overwrite of the first injury:
injured_players[bad_gameplay_id_inds[0]] = 'NO-A.Trautman'

# removing injury updates:
delete_gameplay_ids = [list(inj_play_dict.keys())[bad_ind] for bad_ind in bad_gameplay_id_inds[1:4]]

for gameplay in delete_gameplay_ids:
    del inj_play_dict[gameplay]


['(11:23) Prior to the play, an official timeout for injury was taken as 82-A.Trautman fell down when he was in motion.  He was assisted off the field, and his return is QUESTIONABLE. (Shotgun) A.Kamara left tackle to CIN 16 for 4 yards (G.Pratt, T.Hendrickson).', '(3:14) R.White right tackle to TB 46 for 4 yards (M.Hooker). Injury Update: Dak Prescott has a right hand injury; his return is questionable. Tarell Basham has a thigh injury; his return is questionable.', '(:19) T.Pollard up the middle to TB 45 for 9 yards (D.White). Injury Update: Jayron Kearse has a knee injury; his return is questionable', '(2:24) (Shotgun) E.Elliott up the middle to WAS 45 for 1 yard (D.Payne). Injury Update: Noah Brown has a neck injury; his return is questionable.']


In [109]:
# now that we've removed three plays, we have three fewer injury plays from above
# we also need to rewrite the injured_players list to remove the np.nans

injured_players = [player for player in injured_players if not pd.isnull(player)]

In [110]:
# now we have equal lengths:
print(len(inj_play_dict))
print(len(injured_players))

300
300


In [142]:
# create new dictionary with gameplay:injured player name
injured_players_dict = dict(zip(inj_play_dict.keys(), injured_players))

In [149]:
players = pd.read_csv('../data/provided_data/players.csv')

In [112]:
players.head()

,nflId,height,weight,birthDate,collegeName,position,displayName
0,25511,6-4,225,1977-08-03,Michigan,QB,Tom Brady
1,29550,6-4,328,1982-01-22,Arkansas,T,Jason Peters
2,29851,6-2,225,1983-12-02,California,QB,Aaron Rodgers
3,30842,6-6,267,1984-05-19,UCLA,TE,Marcedes Lewis
4,33084,6-4,217,1985-05-17,Boston College,QB,Matt Ryan


In [121]:
first_pass = players['displayName'].apply(lambda x: x.split(' '))

In [122]:
first_pass

0              [Tom, Brady]
1           [Jason, Peters]
2          [Aaron, Rodgers]
3         [Marcedes, Lewis]
4              [Matt, Ryan]
               ...         
1678      [Ryder, Anderson]
1679         [Jake, Hummel]
1680        [Prince, Emili]
1681    [Ja'Marcus, Ingram]
1682         [C.J., Brewer]
Name: displayName, Length: 1683, dtype: object

In [123]:
# manually fix cases where a player has two paces in their name
first_pass_lengths = [len(combo) for combo in first_pass]
multiple_spaces_inds = [i for i, length in enumerate(first_pass_lengths) if length != 2]
multiple_spaces_names = players['displayName'][multiple_spaces_inds]

In [124]:
multiple_spaces_names

78             Greg Van Roten
139              Kyle Van Noy
396            Robbie  Chosen
578      Leighton Vander Esch
685     Equanimeous St. Brown
876         Andrew Van Ginkel
1143      Prince Tega Wanogho
1321        Amon-Ra St. Brown
1391           Cole Van Lanen
Name: displayName, dtype: object

In [126]:
list(multiple_spaces_names.index)

[78, 139, 396, 578, 685, 876, 1143, 1321, 1391]

In [131]:
corrections = []
for ind in list(multiple_spaces_names.index):
    splits = multiple_spaces_names[ind].split(' ')
    corrections.append([splits[0], ' '.join([name for name in splits[1:] if name != ''])])

In [132]:
corrections

[['Greg', 'Van Roten'],
 ['Kyle', 'Van Noy'],
 ['Robbie', 'Chosen'],
 ['Leighton', 'Vander Esch'],
 ['Equanimeous', 'St. Brown'],
 ['Andrew', 'Van Ginkel'],
 ['Prince', 'Tega Wanogho'],
 ['Amon-Ra', 'St. Brown'],
 ['Cole', 'Van Lanen']]

In [133]:
for ind in list(multiple_spaces_names.index):
    splits = multiple_spaces_names[ind].split(' ')
    first_pass[ind] = [splits[0], ' '.join([name for name in splits[1:] if name != ''])]

In [136]:
# verify correction worked:
# manually fix cases where a player has two paces in their name
first_pass_lengths = [len(combo) for combo in first_pass]
len(set(first_pass_lengths))

1

In [137]:
first_names = [name[0] for name in first_pass]
last_names = [name[1] for name in first_pass]

In [150]:
players.insert(loc = players.shape[1], value = first_names, column = 'first_name')
players.insert(loc = players.shape[1], value = last_names, column = 'last_name')

In [151]:
players.head()

,nflId,height,weight,birthDate,collegeName,position,displayName,first_name,last_name
0,25511,6-4,225,1977-08-03,Michigan,QB,Tom Brady,Tom,Brady
1,29550,6-4,328,1982-01-22,Arkansas,T,Jason Peters,Jason,Peters
2,29851,6-2,225,1983-12-02,California,QB,Aaron Rodgers,Aaron,Rodgers
3,30842,6-6,267,1984-05-19,UCLA,TE,Marcedes Lewis,Marcedes,Lewis
4,33084,6-4,217,1985-05-17,Boston College,QB,Matt Ryan,Matt,Ryan


In [147]:
last_name = 'TEN-O.Adeniyi'.split('.')[1]

In [171]:
first_initial = 'J'

In [159]:
temp_id = list(players['nflId'].loc[players['last_name'] == 'Peters'].values)

In [162]:
temp_id

[29550, 35522, 42361]

In [168]:
first_names = [players['first_name'].loc[players['nflId'] == ind].values[0] for ind in temp_id]

In [175]:
first_names

['Jason', 'Corey', 'Marcus']

In [176]:
first_init_matches = [ind for ind, firstname in enumerate(first_names) if firstname[0] == first_initial]

In [177]:
first_init_matches

[0]

In [179]:
last_name = 'Peters'

In [181]:
players['nflId'].loc[(players['first_name'] == first_names[first_init_matches[0]]) & (players['last_name'] == last_name)].values[0]

29550

In [154]:
players['nflId'].loc[players['last_name'] == last_name].values

46457

In [183]:
matched_nflIds = []

for player in injured_players_dict.values():
    print(f'player = {player}')
    last_name = player.split('.')[1]
    first_initial = player.split('.')[0][-1]

    temp_id = list(players['nflId'].loc[players['last_name'] == last_name].values)

    # have to account for duplicate last names

    if len(temp_id) == 1:
        matched_nflIds.append(temp_id[0])
    else: # look to match first initials if there are multiple of the same last name
        first_names = [players['first_name'].loc[players['nflId'] == ind].values[0] for ind in temp_id]
        first_init_matches = [ind for ind, firstname in enumerate(first_names) if firstname[0] == first_initial]

        # if there's only one first name match, append the correct nflId
        # otherwise we'll have to manually overwrite
        if len(first_init_matches) == 1:
            matched_nflIds.append(players['nflId'].loc[(players['first_name'] == first_names[first_init_matches[0]]) & (players['last_name'] == last_name)].values[0])
        else:
            matched_nflIds.append(np.nan)



        
        
    

player = TEN-O.Adeniyi
player = TEN-T.Burks
player = HST-M.Collins
player = KC-C.Jones
player = JAX-R.Jenkins
player = IND-N.Hines
player = TB-M.Edwards
player = DAL-C.McGovern
player = BLT-M.Moses
player = DAL-Z.Martin
player = TB-C.Brate
player = ATL-A.Ogundeji
player = NYG-L.Williams
player = ATL-A.Terrell
player = BUF-J.Kumerow
player = PIT-J.Pierre
player = WAS-J.Allen
player = CAR-D.Jackson
player = WAS-D.Forrest
player = BUF-S.Diggs
player = BLT-S.Means
player = TEN-A.Hooker
player = WAS-D.Payne
player = BUF-C.Benford
player = ARZ-M.Wilson
player = CIN-J.Williams
player = TEN-K.Fulton
player = TB-C.Davis
player = SF-B.Aiyuk
player = NYG-N.Williams
player = NE-D.Harris
player = MIN-Z.Smith
player = BLT-J.Bynes
player = IND-J.Blackmon
player = BUF-D.Jackson
player = NYJ-K.Alexander
player = PIT-C.Heyward
player = NYG-A.Jackson
player = NYJ-J.Johnson
player = ATL-M.Walker
player = DET-D.Elliott
player = DAL-E.Elliott
player = PIT-M.Cole
player = LA-J.Noteboom
player = GB-E.Stokes
p

IndexError: list index out of range

In [ ]:
injured_players_nflIds = 

In [ ]:
# i think we'd have to subset by injured players involved in the tackle

In [9]:
# what happens when it's a throw? do we care about distance to receiver who caught it or distance to qb?
# i think since this focuses on tackling, we can filter out any play where a tackle wasn't made
# this will include all incompletions
# but, at the same time, we also care about when a tackle was broken and someone ran out of bounds
# so in this sense, we would just filter out incomplete passes
# actually, we can just keep all plays that have a 
tracking_df['event'].value_counts()

event
first_contact                242262
tackle                       230924
ball_snap                    145408
handoff                      135381
pass_outcome_caught          130087
pass_arrived                 113825
out_of_bounds                 41629
run                           21804
man_in_motion                  9683
touchdown                      9523
play_action                    6463
shift                          3358
qb_slide                       3158
fumble                         2415
pass_forward                   1899
snap_direct                    1150
lateral                         618
line_set                        552
pass_shovel                     414
autoevent_passforward           360
autoevent_passinterrupted       324
qb_sack                         230
autoevent_ballsnap              162
fumble_defense_recovered        115
penalty_flag                     93
fumble_offense_recovered         69
safety                           69
run_pass_option       

In [12]:
# we should also parse all of the play descriptions for fun, at some point

In [14]:
plays.loc[plays['passResult'] == 'R'].head()

,gameId,playId,ballCarrierId,ballCarrierDisplayName,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,passResult,passLength,penaltyYards,prePenaltyPlayResult,playResult,playNullifiedByPenalty,absoluteYardlineNumber,offenseFormation,defendersInTheBox,passProbability,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPoints,expectedPointsAdded,foulName1,foulName2,foulNFLId1,foulNFLId2
6,2022100207,2933,38538,Ryan Tannehill,(13:31) (Shotgun) R.Tannehill scrambles up the...,4,3,5,TEN,IND,TEN,25,13:31,17,24,R,NaN,NaN,3,3,N,85,EMPTY,4.0,0.973272,0.230881,0.769119,0.027645,-0.027645,0.137272,-1.269624,NaN,NaN,NaN,NaN
11,2022110606,2433,53431,Zach Wilson,(4:47) (Shotgun) Z.Wilson scrambles right end ...,3,3,3,NYJ,BUF,BUF,12,4:47,10,14,R,NaN,NaN,5,5,N,22,SHOTGUN,6.0,0.880266,0.268702,0.731298,0.038116,-0.038116,4.453594,1.200647,NaN,NaN,NaN,NaN
51,2022091811,3698,47784,Kyler Murray,(2:28) K.Murray scrambles right end pushed ob ...,4,4,1,ARI,LV,LV,43,2:28,23,15,R,NaN,NaN,2,2,N,53,I_FORM,8.0,0.331695,0.896168,0.103832,-0.020265,0.020265,1.313770,1.903174,NaN,NaN,NaN,NaN
81,2022092510,3686,47784,Kyler Murray,"(4:21) (No Huddle, Shotgun) K.Murray scrambles...",4,1,10,ARI,LA,ARI,38,4:21,9,20,R,NaN,NaN,2,2,N,48,SHOTGUN,5.0,0.905591,0.042217,0.957783,-0.016654,0.016654,1.599737,-0.510211,NaN,NaN,NaN,NaN
87,2022091801,428,43380,Jacoby Brissett,(7:39) M.Dunn reported in as eligible. J.Bris...,1,2,7,CLE,NYJ,NYJ,20,7:39,0,0,R,NaN,NaN,4,4,N,30,SINGLEBACK,7.0,0.713875,0.677524,0.322476,-0.001905,0.001905,4.457965,-0.072753,NaN,NaN,NaN,NaN
